In [1]:
import pandas as pd
import re
from pyaspeller import Word
from spellchecker import SpellChecker
from tqdm import tqdm
from gliner import GLiNER


/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1", model_kwargs={'tokenizer': 'spacy'})

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 1937.99it/s]
/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/.venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
model = GLiNER.from_pretrained("/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/models/Gliner_multi", 
                               load_onnx_model=True, 
                               load_tokenizer=True, 
                               onnx_model_file="onnx/model_q4f16.onnx",
                               model_kwargs={'tokenizer': 'spacy'})

2025-09-21 00:31:23.275060476 [W:onnxruntime:, constant_folding.cc:278 ApplyImpl] Could not find a CPU kernel and hence can't constant fold ReduceMean node '/token_rep_layer/bert_layer/model/encoder/LayerNorm/ReduceMean'
2025-09-21 00:31:23.412819234 [W:onnxruntime:, constant_folding.cc:278 ApplyImpl] Could not find a CPU kernel and hence can't constant fold ReduceMean node '/token_rep_layer/bert_layer/model/encoder/LayerNorm/ReduceMean'
2025-09-21 00:31:23.424090959 [W:onnxruntime:, constant_folding.cc:278 ApplyImpl] Could not find a CPU kernel and hence can't constant fold ReduceMean node '/token_rep_layer/bert_layer/model/encoder/LayerNorm/ReduceMean'


In [17]:
import numpy as np
import onnxruntime as ort

# Load session (choose providers if you want GPU)
session = ort.InferenceSession("/home/mikhail/Documents/Хакатоны/X5_ner_MiLky_way/data/onnx/model_fp16.onnx", providers=["CPUExecutionProvider"])

2025-09-20 23:33:55.406200966 [W:onnxruntime:, constant_folding.cc:278 ApplyImpl] Could not find a CPU kernel and hence can't constant fold ReduceMean node '/token_rep_layer/bert_layer/model/encoder/LayerNorm/ReduceMean'
2025-09-20 23:33:55.565176274 [W:onnxruntime:, constant_folding.cc:278 ApplyImpl] Could not find a CPU kernel and hence can't constant fold ReduceMean node '/token_rep_layer/bert_layer/model/encoder/LayerNorm/ReduceMean'
2025-09-20 23:33:55.579880579 [W:onnxruntime:, constant_folding.cc:278 ApplyImpl] Could not find a CPU kernel and hence can't constant fold ReduceMean node '/token_rep_layer/bert_layer/model/encoder/LayerNorm/ReduceMean'


## data

In [3]:
df_test = pd.read_csv('../data/submission.csv', sep=';')

In [4]:
df_test

,sample,annotation
0,форма для выпечки,"[(0, 5, 'B-TYPE'), (6, 9, 'I-TYPE'), (10, 17, ..."
1,фарш свиной,"[(0, 4, 'B-TYPE'), (5, 11, 'I-TYPE')]"
2,сок ананасовый без сахара,"[(0, 3, 'B-TYPE'), (4, 14, 'I-TYPE'), (15, 18,..."
3,еринги,"[(0, 6, 'B-TYPE')]"
4,молооко,"[(0, 7, 'B-TYPE')]"
...,...,...
4995,milkywa,"[(0, 7, 'B-BRAND')]"
4996,очиститель для унитаза,"[(0, 10, 'B-TYPE'), (11, 14, 'I-TYPE'), (15, 2..."
4997,арбузные,"[(0, 8, 'B-TYPE')]"
4998,кашы,"[(0, 4, 'B-TYPE')]"


## spellchecker

In [30]:
samples = df_test['sample'].sample(20, random_state=32).tolist()
samples

['ууа',
 'шоколо',
 'горчича',
 'детские',
 'творог моч цена',
 'сам перемена',
 'тортидья',
 'нафинад',
 'лмона',
 'чай беллини',
 'сельдь oce',
 'салат черепаха',
 'молокоз',
 'ореховая смесь смесь',
 'arco',
 'дезоторант',
 'бул',
 'кроха панам',
 'картофелб',
 'подушечки хрутка']

заметка:
* если слово иностранное, лучше оставлять так как есть
* маппер для ориг.запроса <> исправленной версии, чтобы правильно определять начало-конец сущности

In [ ]:
splitter = lambda x: re.split(r'[ \-.,?]+', x)

for phrase in samples:
    print(f'initial search: {phrase}')
    splitted_phrase = splitter(phrase)
    for wrd in splitted_phrase:
        print(f'word: {wrd}')
        check = Word(wrd)
        print(f'variants: {check.variants}')
        print('******'*10)

## SpellChecker

In [42]:
spell = SpellChecker(language='ru')

In [ ]:
# find those words that may be misspelled
misspelled = spell.unknown(samples)

for word in misspelled:
    print(word)
    # Get the one `most likely` answer
    print(spell.correction(word))
    # Get a list of `likely` options
    print(spell.candidates(word))
    print('****'*30)

## GLiner

In [75]:
samples = df_test.iloc[4640:4660]
samples

,sample,annotation
4640,телец ветчина,"[(0, 5, 'B-TYPE'), (6, 13, 'I-TYPE')]"
4641,говяжий фарш,"[(0, 7, 'B-TYPE'), (8, 12, 'I-TYPE')]"
4642,напольное покрыти,"[(0, 9, 'B-TYPE'), (10, 17, 'I-TYPE')]"
4643,щетки season,"[(0, 5, 'B-TYPE'), (6, 12, 'B-BRAND')]"
4644,прочие фруктовые,"[(0, 6, 'B-TYPE'), (7, 16, 'I-TYPE')]"
4645,ментос,"[(0, 6, 'B-TYPE')]"
4646,клбаса,"[(0, 6, 'B-TYPE')]"
4647,салат черепаха,"[(0, 5, 'B-TYPE'), (6, 14, 'I-TYPE')]"
4648,вода люкс газированная,"[(0, 4, 'B-TYPE'), (5, 9, 'I-TYPE'), (10, 22, ..."
4649,"молоко 1,5 %","[(0, 6, 'B-TYPE'), (7, 10, 'B-VOLUME')]"


In [5]:
def to_bio_spans(entities, gap_tolerance=1):
    # defensive: remove items missing required keys
    ents = [e for e in entities if 'start' in e and 'end' in e and 'label' in e]
    # sort by start
    ents.sort(key=lambda x: (x['start'], x['end']))

    bio = []
    prev_end = None
    prev_label = None

    for e in ents:
        s = e['start']
        t = e['end']
        lbl = e['label'].upper()  # keep case consistent; remove .upper() if you want original case
        # print(prev_label, lbl, s, prev_end)
        if prev_label is not None and lbl == prev_label and (s - prev_end) <= gap_tolerance:
            prefix = 'I-'
        else:
            prefix = 'B-'

        bio.append((s, t, f"{prefix}{lbl}"))

        prev_end = t
        prev_label = lbl

    return bio

In [6]:
labels = ["бренд", "категория товара", "объем", "проценты"]
# labels = ["BRAND", "TYPE", "PERCENT", "VOLUME"]

In [7]:
r = model.predict_entities('мясо свинина индейка', labels, threshold=0.01)
to_bio_spans(r)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[(0, 4, 'B-КАТЕГОРИЯ ТОВАРА'),
 (5, 12, 'I-КАТЕГОРИЯ ТОВАРА'),
 (13, 20, 'I-КАТЕГОРИЯ ТОВАРА')]

In [ ]:
all_spans = []
for i, row in tqdm(df_test.iterrows()):
    # print(f'text: {row['sample']}')
    # print(f'annotation: {row['annotation']}')
    try:
        entities = model.predict_entities(row['sample'], labels, threshold=0.01)
        all_spans.append(to_bio_spans(entities))
    except:
        all_spans.append([])
    # for entity in entities:
    #     print(entity["text"], "=>", entity["label"])
    # print(entities)
    # print(to_bio_spans(entities))
    # print('\n\n' + '#'*50 + '\n\n')


295it [00:45,  5.60it/s]

In [80]:
to_bio_spans(entities)

[(0, 7, 'B-БРЕНД')]

In [18]:
df_test['spans'] = all_spans

In [19]:
df_test

,sample,annotation,spans
0,форма для выпечки,"[(0, 5, 'B-TYPE'), (6, 9, 'I-TYPE'), (10, 17, ...","[(0, 17, B-КАТЕГОРИЯ ТОВАРА)]"
1,фарш свиной,"[(0, 4, 'B-TYPE'), (5, 11, 'I-TYPE')]","[(0, 11, B-КАТЕГОРИЯ ТОВАРА)]"
2,сок ананасовый без сахара,"[(0, 3, 'B-TYPE'), (4, 14, 'I-TYPE'), (15, 18,...","[(0, 14, B-КАТЕГОРИЯ ТОВАРА), (15, 25, I-КАТЕГ..."
3,еринги,"[(0, 6, 'B-TYPE')]","[(0, 6, B-БРЕНД)]"
4,молооко,"[(0, 7, 'B-TYPE')]","[(0, 7, B-КАТЕГОРИЯ ТОВАРА)]"
...,...,...,...
4995,milkywa,"[(0, 7, 'B-BRAND')]","[(0, 7, B-БРЕНД)]"
4996,очиститель для унитаза,"[(0, 10, 'B-TYPE'), (11, 14, 'I-TYPE'), (15, 2...","[(0, 22, B-КАТЕГОРИЯ ТОВАРА)]"
4997,арбузные,"[(0, 8, 'B-TYPE')]","[(0, 8, B-КАТЕГОРИЯ ТОВАРА)]"
4998,кашы,"[(0, 4, 'B-TYPE')]","[(0, 4, B-КАТЕГОРИЯ ТОВАРА)]"


In [24]:
# df_test['new_spans'] = 
df_test['spans'].astype(object).astype(str).replace({'БРЕНД':'BRAND',
                          'КАТЕГОРИЯ ТОВАРА': 'TYPE',
                          'ОБЪЕМ': 'VOLUME',
                          'ПРОЦЕНТЫ': 'PERCENT'})

# labels = ["бренд", "категория товара", "объем", "проценты"]
# # labels = ["BRAND", "TYPE", "PERCENT", "VOLUME"]

0                         [(0, 17, 'B-КАТЕГОРИЯ ТОВАРА')]
1                         [(0, 11, 'B-КАТЕГОРИЯ ТОВАРА')]
2       [(0, 14, 'B-КАТЕГОРИЯ ТОВАРА'), (15, 25, 'I-КА...
3                                     [(0, 6, 'B-БРЕНД')]
4                          [(0, 7, 'B-КАТЕГОРИЯ ТОВАРА')]
                              ...                        
4995                                  [(0, 7, 'B-БРЕНД')]
4996                      [(0, 22, 'B-КАТЕГОРИЯ ТОВАРА')]
4997                       [(0, 8, 'B-КАТЕГОРИЯ ТОВАРА')]
4998                       [(0, 4, 'B-КАТЕГОРИЯ ТОВАРА')]
4999                       [(0, 6, 'B-КАТЕГОРИЯ ТОВАРА')]
Name: spans, Length: 5000, dtype: object

In [29]:
mapping = {
    'БРЕНД': 'BRAND',
    'КАТЕГОРИЯ ТОВАРА': 'TYPE',
    'ОБЪЕМ': 'VOLUME',
    'ПРОЦЕНТЫ': 'PERCENT'
}

# Function to replace inside spans
def replace_spans(spans):
    new_spans = []
    for start, end, label in spans:
        if "-" in label:  # e.g. "B-КАТЕГОРИЯ ТОВАРА"
            prefix, key = label.split("-", 1)
            key = mapping.get(key, key)  # replace if in dict
            new_spans.append((start, end, f"{prefix}-{key}"))
        else:
            # If no prefix, just map
            new_spans.append((start, end, mapping.get(label, label)))
    return new_spans

In [30]:
df_test["new_spans"] = df_test["spans"].apply(replace_spans)

In [31]:
df_test

,sample,annotation,spans,new_spans
0,форма для выпечки,"[(0, 5, 'B-TYPE'), (6, 9, 'I-TYPE'), (10, 17, ...","[(0, 17, B-КАТЕГОРИЯ ТОВАРА)]","[(0, 17, B-TYPE)]"
1,фарш свиной,"[(0, 4, 'B-TYPE'), (5, 11, 'I-TYPE')]","[(0, 11, B-КАТЕГОРИЯ ТОВАРА)]","[(0, 11, B-TYPE)]"
2,сок ананасовый без сахара,"[(0, 3, 'B-TYPE'), (4, 14, 'I-TYPE'), (15, 18,...","[(0, 14, B-КАТЕГОРИЯ ТОВАРА), (15, 25, I-КАТЕГ...","[(0, 14, B-TYPE), (15, 25, I-TYPE)]"
3,еринги,"[(0, 6, 'B-TYPE')]","[(0, 6, B-БРЕНД)]","[(0, 6, B-BRAND)]"
4,молооко,"[(0, 7, 'B-TYPE')]","[(0, 7, B-КАТЕГОРИЯ ТОВАРА)]","[(0, 7, B-TYPE)]"
...,...,...,...,...
4995,milkywa,"[(0, 7, 'B-BRAND')]","[(0, 7, B-БРЕНД)]","[(0, 7, B-BRAND)]"
4996,очиститель для унитаза,"[(0, 10, 'B-TYPE'), (11, 14, 'I-TYPE'), (15, 2...","[(0, 22, B-КАТЕГОРИЯ ТОВАРА)]","[(0, 22, B-TYPE)]"
4997,арбузные,"[(0, 8, 'B-TYPE')]","[(0, 8, B-КАТЕГОРИЯ ТОВАРА)]","[(0, 8, B-TYPE)]"
4998,кашы,"[(0, 4, 'B-TYPE')]","[(0, 4, B-КАТЕГОРИЯ ТОВАРА)]","[(0, 4, B-TYPE)]"


In [32]:
df_new = df_test.copy(deep=True)

In [33]:
df_new = df_new.drop(['annotation', 'spans'], axis=1)

In [38]:
df_new = df_new.rename(columns={'new_spans': 'annotation'})

In [39]:
df_new.to_csv('../data/submissions/submission.csv', index=False, sep=';')